**AI 노바투스과정**

Week 4: 딥러닝 기초 II (2021-08-27)

실습 #3 [**심화 실습**] : 순환 신경망 학습 실습하기 (텍스트 데이터 생성하기)

In [ ]:
### 필요 패키지 불러오기

import numpy as np
import tensorflow.keras as keras
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

## 1. 데이터셋 준비

In [ ]:
# 데이터셋 받아와서 charcter 의 모음인 string 으로 저장하기

shakespeare_url = "https://homl.info/shakespeare" 
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
  shakespeare_text = f.read()

In [ ]:
# 데이터 살피기

print(shakespeare_text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [ ]:
# 각 글자를 정수로 코딩하기

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text)
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1

In [ ]:
# tokenizer 사용해보기

tokenizer.texts_to_sequences(["Apple"])


[[5, 23, 23, 12, 2]]

In [ ]:
tokenizer.sequences_to_texts([[5, 23, 23, 12, 2]])

['a p p l e']

In [ ]:
# 서로 다른 글자의 개수

max_id=len(tokenizer.word_index)

In [ ]:
# 전체 글자의 개수 

dataset_size=tokenizer.document_count

In [ ]:
tokenizer.word_index

{'\n': 11,
 ' ': 1,
 '!': 31,
 '$': 39,
 '&': 38,
 "'": 28,
 ',': 18,
 '-': 32,
 '.': 27,
 '3': 37,
 ':': 24,
 ';': 29,
 '?': 30,
 'a': 5,
 'b': 22,
 'c': 19,
 'd': 13,
 'e': 2,
 'f': 20,
 'g': 21,
 'h': 7,
 'i': 6,
 'j': 33,
 'k': 25,
 'l': 12,
 'm': 15,
 'n': 10,
 'o': 4,
 'p': 23,
 'q': 34,
 'r': 9,
 's': 8,
 't': 3,
 'u': 14,
 'v': 26,
 'w': 17,
 'x': 35,
 'y': 16,
 'z': 36}

In [ ]:
#학습용 데이터셋 분리하기

train_size = dataset_size * 40 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [ ]:
# window() 함수 이용하여 하나의 긴 글을 길이가 101 인 여러 구간으로 분리

n_steps = 100
window_length = n_steps + 1 
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [ ]:
# 구간들을 셔플 (shuffle) 한 후 batch 로 나누고 첫 100 개 글자와 뒤 100 개 글자를 분리

batch_size = 10000
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))



In [ ]:
# 각 정수 값을 one-hot 벡터로 바꾸기

dataset = dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

dataset = dataset.prefetch(1) # 데이터 prefetching

## 2. GRU 모형 만들고 학습하기

In [ ]:
# gpu 설정
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
with tf.device('/device:GPU:0'):
  model = keras.models.Sequential()
  model.add(keras.layers.GRU(64, return_sequences=True, input_shape=[None, max_id],
  dropout=0.2, recurrent_dropout=0))
  model.add(keras.layers.GRU(64, return_sequences=True, dropout=0.2, recurrent_dropout=0))
  model.add(keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation="softmax")))

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=10)

Epoch 1/10
45/45 [==============================] - 69s 1s/step - loss: 3.2892
Epoch 2/10
45/45 [==============================] - 61s 1s/step - loss: 3.0468
Epoch 3/10
45/45 [==============================] - 61s 1s/step - loss: 2.8960
Epoch 4/10
45/45 [==============================] - 61s 1s/step - loss: 2.6699
Epoch 5/10
45/45 [==============================] - 62s 1s/step - loss: 2.5557
Epoch 6/10
45/45 [==============================] - 61s 1s/step - loss: 2.4966
Epoch 7/10
45/45 [==============================] - 61s 1s/step - loss: 2.4518
Epoch 8/10
45/45 [==============================] - 61s 1s/step - loss: 2.4128
Epoch 9/10
45/45 [==============================] - 61s 1s/step - loss: 2.3771
Epoch 10/10
45/45 [==============================] - 61s 1s/step - loss: 2.3453


##3. 글자 생성하기

In [ ]:
# 입력 데이터를 숫자 & one-hot vector 로 변환하는 함수 만들기

def preprocess(texts):
  X = np.array(tokenizer.texts_to_sequences(texts)) - 1
  return tf.one_hot(X, max_id)

In [ ]:
import numpy as np
# "How are yo" 예측
X_new = preprocess(["How are yo"])
y_proba=model.predict(X_new)
Y_pred=y_proba.argmax(axis=2)
print(Y_pred)
tokenizer.sequences_to_texts(Y_pred+1)[0][-1]

[[ 1 13  0  2  9 12  0  2  3 13]]


'u'

In [ ]:
# "hel" 예측

X_new = preprocess(["hel"])
y_proba=model.predict(X_new)
Y_pred=y_proba.argmax(axis=2)
print(Y_pred)
tokenizer.sequences_to_texts(Y_pred+1)[0][-1]

[[ 1  0 11]]


'l'

In [ ]:
tokenizer.sequences_to_texts(Y_pred+1)

['e   l']

In [ ]:
tokenizer.sequences_to_texts(Y_pred+1)[0]

'e   l'

In [ ]:
tokenizer.sequences_to_texts(Y_pred+1)[0][-1]

'l'

## 4. 가짜 글 생성하기

In [ ]:
def next_char(text, temperature=1):
  X_new = preprocess([text])
  y_proba = model.predict(X_new)[0, -1:, :]
  rescaled_logits = tf.math.log(y_proba) / temperature
  char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
  return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [ ]:
def complete_text(text, n_chars=50, temperature=1):
  for _ in range(n_chars):
    text += next_char(text, temperature)
  return text

In [ ]:
print(complete_text("t", temperature=0.2))


the sing to the the sond and the the the the he han


In [ ]:
print(complete_text("w", temperature=2))


w.,ifls;frsog wt' hatabftious enimg:oun ham,t:ht ch


## 5. Stateful RNN 모형으로 해보기

In [ ]:
## 학습용 데이터 준비

# ***주의*** 각 입력 시퀀스는 이전 시퀀스와 이어져야 하므로,  
# shift=n_steps 으로 하며, shuffle() 을 사용하지 말것.


dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
dataset = dataset.batch(1)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [ ]:
# 모형의 골격 만들기 (Stateful=True 로 설정)

batch_size = 1
with tf.device('/device:GPU:0'):
  model = keras.models.Sequential()
  model.add( keras.layers.GRU(128, return_sequences=True, stateful=True, dropout=0.2, recurrent_dropout=0,
  batch_input_shape=[batch_size, None, max_id]) )
  model.add( keras.layers.GRU(128, return_sequences=True, stateful=True,
  dropout=0.2, recurrent_dropout=0) )
  model.add( keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation="softmax")))


In [ ]:
# callback 만들기 (다음 epoch 으로 넘어가기 전에 state 재설정하는 callback)

class ResetStatesCallback(keras.callbacks.Callback):
  def on_epoch_begin(self, epoch, logs):
    self.model.reset_states()

In [ ]:
# 모형 컴파일 및 학습하기

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
model.fit(dataset, epochs=20, callbacks=[ResetStatesCallback()])

Epoch 1/20
    224/Unknown - 5s 12ms/step - loss: 2.8645

KeyboardInterrupt: ignored